## <font color="purple"> Les opérations map et reduce </font>

Disposant de la liste suivante, proposer 3 syntaxes pour calculer la liste des carrés de ses éléments :

In [0]:
a = [1, 4, 74, 2]

In [0]:
b = []
for number in a:
    b.append(number ** 2)
print(b)

[1, 16, 5476, 4]


In [0]:
b = [number ** 2 for number in a]
print(b)

[1, 16, 5476, 4]


In [0]:
def f(x):
    return x ** 2

b = list(map(f, a))

print(b)

[1, 16, 5476, 4]


In [0]:
map(lambda x: x**2, a)

On souhaite désormais ajouter récursivement les éléments de cette nouvelle liste. Réaliser cette opération à l'aide de la fonction reduce de functools.

In [0]:
from functools import reduce

def g(x, y):
    return x + y

c = reduce(g, b)
print(c)

5497


map et reduce sont des opérateurs génériques et leur combinaison permet donc de modéliser énormément de problèmes.

Recette de MapReduce :

1. L'ensemble des données à traiter est découpé en plusieurs lots ou sous-ensembles.

2. Dans une première étape, l'étape MAP, l'opérationmap, spécifiée pour notre problème, est appliquée à chaque lot. Cette opération transforme la paire(clé, valeur)représentant le lot en une liste de nouvelles paires(clé, valeur)constituant ainsi des résultats intermédiaires du traitement à effectuer sur les données complètes.

3. Avant d'être envoyés à l'étape REDUCE, les résultats intermédiaires sont regroupés et triés par clé. C'est l'étape de SHUFFLE and SORT.

4. Enfin, l'étape REDUCE consiste à appliquer l'opération reduce, spécifiée pour notre problème, à chaque clé. Elle agrège tous les résultats intermédiaires associés à une même clé et renvoie donc pour chaque clé une valeur unique.

![Schéma MapReduce](../../docs/images/schema_mapreduce.jpeg)

## Word Count

defaultdict(int) est une structure de données fournie par le module collections de Python. Il s'agit d'une sous-classe de dictionnaire qui permet de définir une valeur par défaut pour les clés qui n'existent pas.
Il est possible de lui préciser un type de données par défaut.

In [0]:
from collections import defaultdict

En utilisant int comme type de données par défaut (defaultdict(int)), si une clé n'existe pas dans le dictionnaire, elle sera automatiquement initialisée avec la valeur 0.

Ecrire une fonction Python word_count permettant de renvoyer le defaultdict des paires mots, comptages d'un texte passé en argument. On pourra considérer que les mots sont toujours séparés par des espaces et utiliser la méthode split() pour séparer le texte en mots.

In [0]:
def word_count(text):
    counts = defaultdict(int)
    for word in text.split():
        counts[word.lower()] += 1
    return counts

In [0]:
word_count("ceci est un texte et un test")

Out[10]: defaultdict(int,
            {'ceci': 1, 'est': 1, 'un': 2, 'texte': 1, 'et': 1, 'test': 1})

Dans la suite, nous allons mettre en oeuvre le principe de MapReduce sur les paroles d'une chanson.

Nous allons supposer que nos données d'entrée ont été découpées en différents fragments et qu'une opération de simplification a été appliquée sur chaque fragment pour supprimer les caractères de ponctuation, transformer chaque mot en son singulier ("nos" devient "notre") et ne garder que les mots de plus de 3 caractères.

Voici ces morceaux :

In [0]:
D1 = {"./lot1.txt" : "jour lève notre grisaille"}
D2 = {"./lot2.txt" : "trottoir notre ruelle notre tour"}
D3 = {"./lot3.txt" : "jour lève notre envie vous"}
D4 = {"./lot4.txt" : "faire comprendre tous notre tour"}

Ecrire une fonction map s'appliquant sur une paire clef, valeur qui renvoie le comptage des mots dans la valeur (la clef ne sera pas utilisée, ce n'est pas grave).
Par exemple, son appel sur la clef et la valeur de D1 renverra [('jour', 1), ('lève', 1), ('notre', 1), ('grisaille', 1)].

In [0]:
def map_(key, value):
    intermediate = []
    for word in value.split():
        intermediate.append((word, 1))
    return intermediate

In [0]:
key, value = next(iter(D1.items()))
print(map_(key, value))

[('jour', 1), ('lève', 1), ('notre', 1), ('grisaille', 1)]


![Image map](../../docs/images/map.jpeg)

A ce stade, nous allons considérer que nous sommes un peu magicien et que nous sommes capables de regrouper et de trier, par clé commune, les résultats intermédiaires fournis par l'étape MAP. Cela correspond à l'étape SHUFFLE and SORT. Dans la pratique, cette étape est entièrement gérée par le framework d'exécution de MapReduce, de manière distribuée.

![Shuffle and sort](../../docs/images/shuffle_sort.jpeg)

Ecrire une fonction reduce prenant en arguments une clef et une liste de valeurs et retournant un tuple formé de la clef et de la somme des valeurs de la liste. Par exemple, son appel sur ("notre", [1, 1, 1, 1, 1]) renverra 5.

In [0]:
def reduce_(key, values):
    result = 0
    for c in values:
        result += c
    return (key, result)

In [0]:
print(reduce_("notre", [1, 1, 1, 1, 1]))

('notre', 5)


Et voilà, nous avons notre comptage de mots !

![Schéma global](../../docs/images/global.jpeg)

En résumé, MapReduce c'est :

- La généralisation du paradigme de conception d'algorithmes diviser pour régner au cadre distribué.

- Un modèle de programmation reposant sur la combinaison de deux fonctions simples,mapetreduce, inspirées de la programmation fonctionnelle.

- Un framework d'exécution prenant en charge le deploiement et la distribution des calculs sur un cluster.

Le rôle des developpeurs d'applications distribuées, c'est donc de penser en MapReduce :

- Choisir une manière de découper les données afin que l'opération MAP soit parallélisable.

- Choisir la clé à utiliser pour le problème ciblé.

- Écrire le code de la fonction pour l'opération MAP.

- Écrire le code de la fonction pour l'opération REDUCE.